# Piecewise Efficiency: Heat Pump with Variable COP

## User Story

> *You're installing a heat pump for a commercial building. The heat pump's efficiency (COP - Coefficient of Performance) varies with the outdoor temperature: it's more efficient in mild weather than in cold weather. You want to model this realistically to accurately predict operating costs.*

This notebook introduces:

- **Piecewise linear functions**: Approximate non-linear behavior
- **Variable efficiency**: COP changes with operating conditions
- **LinearConverter with segments**: Multiple operating points
- **Piecewise effects**: Non-linear cost curves

## Setup

In [ ]:
import numpy as np
import pandas as pd
import plotly.express as px
import xarray as xr

import flixopt as fx

fx.CONFIG.notebook()

## The Problem: Variable Heat Pump Efficiency

A heat pump's COP (Coefficient of Performance) depends on the temperature difference between source and sink:

- **Mild weather** (10°C outside): COP ≈ 4.5 (1 kWh electricity → 4.5 kWh heat)
- **Cold weather** (-5°C outside): COP ≈ 2.5 (1 kWh electricity → 2.5 kWh heat)

This non-linear relationship can be approximated using piecewise linear segments.

## Define Time Series Data

In [ ]:
# One winter week
timesteps = pd.date_range('2024-01-22', periods=168, freq='h')
hours = np.arange(168)
hour_of_day = hours % 24

# Outdoor temperature: daily cycle with cold nights
temp_base = 2  # Average temp
temp_amplitude = 5  # Daily variation
outdoor_temp = temp_base + temp_amplitude * np.sin((hour_of_day - 6) * np.pi / 12)

# Add day-to-day variation
np.random.seed(789)
daily_offset = np.repeat(np.random.uniform(-3, 3, 7), 24)
outdoor_temp = outdoor_temp + daily_offset

print(f'Temperature range: {outdoor_temp.min():.1f}°C to {outdoor_temp.max():.1f}°C')

In [ ]:
# Heat demand: inversely related to outdoor temp
# Higher demand when colder
heat_demand = 200 - 8 * outdoor_temp  # Simple linear relationship
heat_demand = np.clip(heat_demand, 100, 300)

print(f'Heat demand range: {heat_demand.min():.0f} - {heat_demand.max():.0f} kW')

In [ ]:
# Visualize with plotly - using xarray and faceting
profiles = xr.Dataset(
    {
        'Outdoor Temp [°C]': xr.DataArray(outdoor_temp, dims=['time'], coords={'time': timesteps}),
        'Heat Demand [kW]': xr.DataArray(heat_demand, dims=['time'], coords={'time': timesteps}),
    }
)

df = profiles.to_dataframe().reset_index().melt(id_vars='time', var_name='variable', value_name='value')
fig = px.line(df, x='time', y='value', facet_col='variable', height=300)
fig.update_yaxes(matches=None)
fig.for_each_annotation(lambda a: a.update(text=a.text.split('=')[-1]))
fig.show()

## Calculate Time-Varying COP

The COP depends on outdoor temperature. We use a simplified Carnot-based formula:

In [ ]:
# COP calculation (simplified)
# Real COP ≈ 0.4 * Carnot COP
T_supply = 45 + 273.15  # Supply temperature 45°C in Kelvin
T_source = outdoor_temp + 273.15  # Outdoor temp in Kelvin

carnot_cop = T_supply / (T_supply - T_source)
real_cop = 0.45 * carnot_cop  # Real efficiency factor
real_cop = np.clip(real_cop, 2.0, 5.0)  # Physical limits

print(f'COP range: {real_cop.min():.2f} - {real_cop.max():.2f}')

In [ ]:
# Plot COP vs temperature using plotly
px.scatter(
    x=outdoor_temp,
    y=real_cop,
    title='Heat Pump COP vs Outdoor Temperature',
    labels={'x': 'Outdoor Temperature [°C]', 'y': 'COP'},
    opacity=0.5,
)

## Approach 1: Simple Model with Time-Varying Efficiency

The simplest approach: use time-varying conversion factors directly.

In [ ]:
flow_system = fx.FlowSystem(timesteps)

flow_system.add_elements(
    fx.Bus('Electricity', carrier='electricity'),
    fx.Bus('Heat', carrier='heat'),
    fx.Effect('costs', '€', 'Operating Costs', is_standard=True, is_objective=True),
    # Electricity grid
    fx.Source(
        'Grid',
        outputs=[fx.Flow('Elec', bus='Electricity', size=500, effects_per_flow_hour=0.30)],
    ),
    # Heat pump with time-varying COP
    fx.LinearConverter(
        'HeatPump',
        inputs=[fx.Flow('Elec', bus='Electricity', size=150)],
        outputs=[fx.Flow('Heat', bus='Heat', size=500)],
        # Time-varying conversion factor = COP
        conversion_factors=[{('Elec', 'Heat'): real_cop}],
    ),
    # Building heat demand
    fx.Sink(
        'Building',
        inputs=[fx.Flow('Heat', bus='Heat', size=1, fixed_relative_profile=heat_demand)],
    ),
)

flow_system.optimize(fx.solvers.HighsSolver())
print(f'Total cost: {flow_system.solution["costs"].item():.2f} €')

In [ ]:
flow_system.statistics.plot.balance('Heat')

In [ ]:
flow_system.statistics.plot.balance('Electricity')

## Approach 2: Piecewise Linear Efficiency

For more complex efficiency curves (e.g., part-load efficiency), use `Piecewise`:

In [ ]:
# Define piecewise efficiency for a gas engine
# Efficiency varies with load: lower at part load, optimal at 75% load

# Engine: 100 kW electrical capacity
# Part-load efficiency curve:
#   - 25% load: 32% efficiency
#   - 50% load: 38% efficiency
#   - 75% load: 42% efficiency (optimal)
#   - 100% load: 40% efficiency

# In flixopt, Piecewise defines output as function of input
# For engine: fuel_input → electrical_output

# At various operating points (fuel in kW → elec out kW):
# 25% load: 78 kW fuel → 25 kW elec (η=32%)
# 50% load: 132 kW fuel → 50 kW elec (η=38%)
# 75% load: 179 kW fuel → 75 kW elec (η=42%)
# 100% load: 250 kW fuel → 100 kW elec (η=40%)

piecewise_efficiency = fx.Piecewise(
    fx.Piece(start=(78, 25), end=(132, 50)),  # Segment 1
    fx.Piece(start=(132, 50), end=(179, 75)),  # Segment 2
    fx.Piece(start=(179, 75), end=(250, 100)),  # Segment 3
)

print('Piecewise segments:')
for i, piece in enumerate(piecewise_efficiency.pieces):
    fuel_in = (piece.start[0] + piece.end[0]) / 2
    elec_out = (piece.start[1] + piece.end[1]) / 2
    efficiency = elec_out / fuel_in * 100
    print(f'  Segment {i + 1}: ~{efficiency:.1f}% efficiency')

In [ ]:
# Build system with piecewise efficiency
timesteps_simple = pd.date_range('2024-01-22', periods=48, freq='h')  # 2 days
elec_demand_simple = np.concatenate(
    [
        np.linspace(30, 90, 24),  # Day 1: ramp up
        np.linspace(90, 30, 24),  # Day 2: ramp down
    ]
)

fs_piecewise = fx.FlowSystem(timesteps_simple)

fs_piecewise.add_elements(
    fx.Bus('Gas', carrier='gas'),
    fx.Bus('Electricity', carrier='electricity'),
    fx.Effect('costs', '€', 'Costs', is_standard=True, is_objective=True),
    fx.Source('GasGrid', outputs=[fx.Flow('Gas', bus='Gas', size=500, effects_per_flow_hour=0.05)]),
    # Engine with piecewise efficiency
    fx.LinearConverter(
        'GasEngine',
        inputs=[fx.Flow('Fuel', bus='Gas')],
        outputs=[fx.Flow('Elec', bus='Electricity')],
        piecewise_conversion=piecewise_efficiency,
    ),
    fx.Sink('Load', inputs=[fx.Flow('Elec', bus='Electricity', size=1, fixed_relative_profile=elec_demand_simple)]),
)

fs_piecewise.optimize(fx.solvers.HighsSolver())

In [ ]:
fs_piecewise.statistics.plot.balance('Electricity')

In [ ]:
fs_piecewise.statistics.plot.balance('Gas')

In [ ]:
# Analyze actual efficiency
fuel_used = fs_piecewise.statistics.flow_rates['GasEngine(Fuel)'].values
elec_produced = fs_piecewise.statistics.flow_rates['GasEngine(Elec)'].values

actual_efficiency = np.where(fuel_used > 0, elec_produced / fuel_used * 100, 0)

# Plot using plotly
fig = px.line(
    x=timesteps_simple,
    y=actual_efficiency,
    title='Gas Engine Operating Efficiency',
    labels={'x': 'Time', 'y': 'Efficiency [%]'},
)
fig.update_traces(mode='lines+markers', marker=dict(size=4))
fig.update_yaxes(range=[30, 45])
fig.show()

## Approach 3: Piecewise Investment Costs

Piecewise functions can also model economies of scale in investment:

In [ ]:
# Investment cost curve with economies of scale:
# - Small system (0-100 kW): 1000 €/kW
# - Medium system (100-300 kW): 800 €/kW
# - Large system (300-500 kW): 600 €/kW

# Cumulative costs at breakpoints:
# 100 kW: 100,000 €
# 300 kW: 100,000 + 200*800 = 260,000 €
# 500 kW: 260,000 + 200*600 = 380,000 €

piecewise_invest_costs = fx.PiecewiseEffects(
    {
        'costs': fx.Piecewise(
            fx.Piece(start=(0, 0), end=(100, 100_000)),  # 1000 €/kW
            fx.Piece(start=(100, 100_000), end=(300, 260_000)),  # 800 €/kW
            fx.Piece(start=(300, 260_000), end=(500, 380_000)),  # 600 €/kW
        )
    }
)

print('Investment cost curve:')
print('  0-100 kW: 1000 €/kW')
print('  100-300 kW: 800 €/kW')
print('  300-500 kW: 600 €/kW')

## Key Concepts

### Time-Varying Conversion Factors

For temperature-dependent efficiency, use arrays:
```python
fx.LinearConverter(
    'HeatPump',
    inputs=[fx.Flow('Elec', bus='Electricity', size=150)],
    outputs=[fx.Flow('Heat', bus='Heat', size=500)],
    conversion_factors=[{('Elec', 'Heat'): cop_array}],  # Time-varying
)
```

### Piecewise Linear Functions

For non-linear relationships (part-load efficiency):
```python
fx.Piecewise(
    fx.Piece(start=(input1, output1), end=(input2, output2)),
    fx.Piece(start=(input2, output2), end=(input3, output3)),
    ...
)
```

### When to Use Each Approach

| Approach | Use Case | Complexity |
|----------|----------|------------|
| Time-varying factors | Efficiency depends on external conditions (temp, price) | Low |
| Piecewise conversion | Efficiency depends on load level | Medium |
| Piecewise effects | Non-linear costs (economies of scale) | Medium |

## Summary

You learned how to:

- Model **time-varying efficiency** using conversion factor arrays
- Use **Piecewise** for load-dependent efficiency curves
- Apply **PiecewiseEffects** for non-linear cost functions
- Choose the right approach for your modeling needs

### Next Steps

- **[07-scenarios-and-periods](07-scenarios-and-periods.ipynb)**: Multi-scenario planning
- **[08-large-scale-optimization](08-large-scale-optimization.ipynb)**: Computational efficiency techniques